# Library imports, configurations

In [1]:
import os
import cv2
import subprocess
import seaborn as sns
import s3fs
import pandas as pd
import numpy as np
from scipy.stats import uniform, randint
from PIL import Image
import pyvips
import torch
from joblib import dump
from pathlib import Path
import random
from cloudpathlib import S3Path, S3Client
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.metrics import log_loss, accuracy_score, mean_squared_error, precision_recall_curve
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.utils import class_weight
from sklearn import preprocessing
from sklearn.datasets import make_classification
from sklearn.preprocessing import binarize, LabelEncoder, MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from mlxtend.classifier import StackingClassifier

# import tensorflow as tf
# from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
# from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
# from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# import tensorflow_decision_forests as tfdf

from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

ModuleNotFoundError: No module named 'torch'

In [136]:
Image.MAX_IMAGE_PIXELS = None

In [137]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [138]:
BUCKET = "jplaton/diffusion"
fs.ls(BUCKET)

['jplaton/diffusion/.keep', 'jplaton/diffusion/visio_mel']

# DATA Download

## Metadata and labels

In [139]:
FILE_PATH_TRAIN = "visio_mel/train_labels.csv"
FILE_PATH_TRAIN_S3 = BUCKET + "/" + FILE_PATH_TRAIN

with fs.open(FILE_PATH_TRAIN_S3, mode="rb") as file_in:
    train_labels = pd.read_csv(file_in, sep=",")

In [140]:
FILE_PATH_TRAIN = "visio_mel/train_metadata.csv"
FILE_PATH_TRAIN_S3 = BUCKET + "/" + FILE_PATH_TRAIN

with fs.open(FILE_PATH_TRAIN_S3, mode="rb") as file_in:
    train_metadata = pd.read_csv(file_in, sep=",")

In [141]:
FILE_PATH_TEST = "visio_mel/visiomel_code_execution_development_data/test_labels.csv"
FILE_PATH_TEST_S3 = BUCKET + "/" + FILE_PATH_TEST

with fs.open(FILE_PATH_TEST_S3, mode="rb") as file_in:
    test_labels = pd.read_csv(file_in, sep=",")

In [142]:
FILE_PATH_TEST = "visio_mel/visiomel_code_execution_development_data/test_metadata.csv"
FILE_PATH_TEST_S3 = BUCKET + "/" + FILE_PATH_TEST

with fs.open(FILE_PATH_TEST_S3, mode="rb") as file_in:
    test_metadata = pd.read_csv(file_in, sep=",")

## Images

In [143]:
# select a set of images to plot
SEED = 42
NUM_IMAGES = 3

# we'll use the US url
selected_image_paths = train_metadata.sample(
    random_state=SEED, n=NUM_IMAGES
).us_tif_url.values

In [144]:
def visualize_page(page_num, image_paths):
    fig, axes = plt.subplots(1, len(image_paths), figsize=(10, 10))
    fig.tight_layout()

    for n, image_s3_path in enumerate(image_paths):
        fname = S3Path(image_s3_path).name
        print(f"Downloading {fname}")
        local_file = S3Path(image_s3_path, 
                             client=S3Client(local_cache_dir='/tmp/images', no_sign_request=True)).fspath

        n_frames = Image.open(local_file).n_frames
        img = pyvips.Image.new_from_file(local_file, page=page_num).numpy()

        axes[n].set_title(f"{fname}, page={page_num}\n {img.shape}\n")
        axes[n].imshow(img)

In [ ]:
# let's look at a few high-resolution pages from our selected images
visualize_page(3, selected_image_paths)

In [ ]:
# let's look at low-resolution pages from the same images
visualize_page(8, selected_image_paths)

# Training the model

## Preprocessing
To start, let's do some feature engineering! We'll first take a look at the unique values for each variable in the metadatatrain_metadata.age.unique()

In [145]:
train_metadata.age.unique()

array(['[32:34[', '[46:48[', '[64:66[', '[62:64[', '[90:92[', '[74:76[',
       '[68:70[', '[54:56[', '[80:82[', '[72:74[', '[40:42[', '[58:60[',
       '[86:88[', '[60:62[', '[78:80[', '[66:68[', '[38:40[', '[28:30[',
       '[48:50[', '[70:72[', '[42:44[', '[44:46[', '[30:32[', '[52:54[',
       '[56:58[', '[76:78[', '[50:52[', '[82:84[', '[84:86[', '[26:28[',
       '[18:20[', '[34:36[', '[36:38[', '[92:94[', '[88:90[', '[24:26[',
       '[20:22[', '[94:96[', '[22:24[', '[98:100[', '[16:18['],
      dtype=object)

In [146]:
train_metadata.breslow.unique()

array(['<0.8', '[1 : 2[', '[2 : 4[', '[0.8 : 1[', nan, '>=4'],
      dtype=object)

In [147]:
train_metadata.sex.unique()

array([2, 1])

In [148]:
train_metadata.body_site.unique()

array(['thigh', 'trunc', 'face', 'forearm', 'arm', 'leg', 'hand', nan,
       'foot', 'sole', 'finger', 'neck', 'toe', 'seat', 'scalp', 'nail',
       'trunk', 'lower limb/hip', 'hand/foot/nail', 'head/neck',
       'upper limb/shoulder'], dtype=object)

In [149]:
train_metadata.melanoma_history.unique()

array(['YES', 'NO', nan], dtype=object)

So far, we've learned:

- age is available to us in two-year intervals
- melanoma_history is sometimes nan
- body_site categories are not all at the same level of aggregation - for example, some slides are labeled as being from "hand/foot/nail" while some slides are labeled as from the hand, foot, and nail individually.

Let's do the following:

- convert age to integer values
- convert melanoma_history to dummy variables
- For this first pass, we will not use body_site.

In [153]:
# define an encoder for melanoma_history, body_site 
enc = OneHotEncoder(drop="first", sparse_output=False)
df_enc = train_metadata[['melanoma_history','body_site']].copy()
df_enc['body_site']=train_metadata['body_site'].str.replace(' ','_').str.replace('/','_')
enc.fit(df_enc)
enc.get_feature_names_out()

array(['melanoma_history_YES', 'melanoma_history_nan', 'body_site_face',
       'body_site_finger', 'body_site_foot', 'body_site_forearm',
       'body_site_hand', 'body_site_hand_foot_nail',
       'body_site_head_neck', 'body_site_leg', 'body_site_lower_limb_hip',
       'body_site_nail', 'body_site_neck', 'body_site_scalp',
       'body_site_seat', 'body_site_sole', 'body_site_thigh',
       'body_site_toe', 'body_site_trunc', 'body_site_trunk',
       'body_site_upper_limb_shoulder', 'body_site_nan'], dtype=object)

In [ ]:
# define an encoder for breslow
# enc_l = LabelEncoder()
# enc_l.fit(train_metadata['breslow'])

In [154]:
def preprocess_feats(df=train_metadata):
    feats = df.copy()
    # take the first age in the range and convert to integer
    feats["age_int"] = feats.age.str.slice(1, 3).astype(int)
    feats['body_site']=feats['body_site'].str.replace(' ','_').str.replace('/','_')
    X = pd.concat(
        [
            feats[["age_int", "sex","resolution"]],
            pd.DataFrame(
                enc.transform(feats[["melanoma_history","body_site"]]),
                columns=enc.get_feature_names_out(),
                index=feats.index,
            ),
        ],
        axis=1,
    )

    return X

In [155]:
# preprocess features

X = preprocess_feats(train_metadata)
X.dtypes

age_int                            int64
sex                                int64
resolution                       float64
melanoma_history_YES             float64
melanoma_history_nan             float64
body_site_face                   float64
body_site_finger                 float64
body_site_foot                   float64
body_site_forearm                float64
body_site_hand                   float64
body_site_hand_foot_nail         float64
body_site_head_neck              float64
body_site_leg                    float64
body_site_lower_limb_hip         float64
body_site_nail                   float64
body_site_neck                   float64
body_site_scalp                  float64
body_site_seat                   float64
body_site_sole                   float64
body_site_thigh                  float64
body_site_toe                    float64
body_site_trunc                  float64
body_site_trunk                  float64
body_site_upper_limb_shoulder    float64
body_site_nan   

In [156]:
# take labels from train_labels.csv

y = train_labels.relapse

## Model training and calibration
For a first pass at training, we will have a simple stratified 80-20 split between train and test. The stratified split helps ensure that the proportion of relapse cases are similar between our train and test set and that we are performing our model training on a representative subset of the data.

In [157]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

In [158]:
X_test

,age_int,sex,resolution,melanoma_history_YES,melanoma_history_nan,body_site_face,body_site_finger,body_site_foot,body_site_forearm,body_site_hand,...,body_site_neck,body_site_scalp,body_site_seat,body_site_sole,body_site_thigh,body_site_toe,body_site_trunc,body_site_trunk,body_site_upper_limb_shoulder,body_site_nan
1242,52,2,0.220512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
460,26,2,0.226490,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
614,56,2,0.242797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
317,70,2,0.264384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
263,80,1,0.250000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,50,2,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
174,46,1,0.250000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1045,66,1,0.263223,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
905,42,2,0.250000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [159]:
y_train.value_counts(normalize=True)

relapse
0    0.841566
1    0.158434
Name: proportion, dtype: float64

In [160]:
y_test.value_counts(normalize=True)

relapse
0    0.840149
1    0.159851
Name: proportion, dtype: float64

In [161]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

### Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=SEED)

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

search = RandomizedSearchCV(rf, param_distributions=params, random_state=42, n_iter=200, cv=5, verbose=1, n_jobs=1, return_train_score=True)

search.fit(X_train,y_train)

report_best_scores(search.cv_results_,n_top=1)

In [ ]:
# fit a calibrated random forest model

rf = RandomForestClassifier(random_state=SEED,n_estimators = 200,min_samples_split = 2,min_samples_leaf = 4,
max_features = 'sqrt', max_depth = 2, bootstrap = True)
calibrated_rf = CalibratedClassifierCV(rf, method="sigmoid", cv=5)
calibrated_rf.fit(X_train, y_train)

### Gradient Boosting

In [ ]:
modelGB = GradientBoostingClassifier()

params = {"learning_rate": uniform(),
            "subsample"    : uniform(),
                  "n_estimators" : randint(100, 500),
                  "max_depth"    : randint(2, 10)
                 }

search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, n_iter=200, cv=5, verbose=1, n_jobs=1, return_train_score=True)

search.fit(X_train,y_train)

report_best_scores(search.cv_results_,n_top=1)

In [ ]:
modelGB = GradientBoostingClassifier(learning_rate = 0.01083765148029836,n_estimators=175,
max_depth = 2, subsample = 0.3549051904627206)
calibrated_GB = CalibratedClassifierCV(modelGB, method="sigmoid", cv=5)
modelGB_fit = calibrated_GB.fit(X_train,y_train)

### XGBoost

In [179]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic")

params = {
    "max_depth": randint(2, 10), # default 3
    "n_estimators": randint(100, 200), # default 100
    "learning_rate": uniform(0.01, 0.3), # default 0.1 
    "colsample_bytree": uniform(0.7, 0.3),
    "subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, n_iter=200, cv=5, verbose=1, n_jobs=1, return_train_score=True)

search.fit(X_train,y_train)

report_best_scores(search.cv_results_,n_top=1)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Model with rank: 1
Mean validation score: 0.842 (std: 0.002)
Parameters: {'colsample_bytree': 0.7580363202534581, 'learning_rate': 0.023800792606525824, 'max_depth': 2, 'n_estimators': 193, 'subsample': 0.9421842336044028}



/opt/mamba/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
380 fits failed out of a total of 1000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/mamba/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/mamba/lib/python3.10/site-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/opt/mamba/lib/python3.10/site-packages/xgboost/sklearn.py", line 1490, in fit
    self._Booster = train(
  File "/opt/mamba/lib/python3.10/site-packages/xgboost/core.py", line 620, in inner_f


In [180]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=SEED, max_depth = 2, n_estimators = 193, learning_rate = 0.023800792606525824,
colsample_bytree=0.7580363202534581, subsample=0.9421842336044028)
calibrated_xgb = CalibratedClassifierCV(xgb_model, method="sigmoid", cv=5)
xbg_model_fit = calibrated_xgb.fit(X_train,y_train)

### Neural Network (Tensorflow)

In [ ]:
# Create input functions

batch_size = 100
train_steps = 1000


def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    return dataset.shuffle(1000).repeat().batch(batch_size)

def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset

In [ ]:
# Define Tensorflow feature columns
feature_columns = [tf.feature_column.numeric_column(x) for x in X.columns]

In [ ]:
# Instantiate an Estimator

# Build a DNN with 2 hidden layers and 10 nodes in each hidden layer.
model = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                    hidden_units=[1024, 512, 256],
                                    optimizer=tf.keras.optimizers.legacy.Adam(lr=0.0001))

### Train, Evaluate, and Predict
Now that we have an Estimator object, we can call methods to do the following:

Train the model.
Evaluate the trained model.
Use the trained model to make predictions.

In [ ]:
#Train the model
#The steps argument tells the method to stop training after a number of training steps.

model.train(input_fn=lambda:train_input_fn(X_train, y_train, batch_size), steps=train_steps)

In [ ]:
#Evaluate the trained model
#Now that the model has been trained, we can get some statistics on its performance. The following code block evaluates the accuracy of the trained model on the test data.
# Evaluate the model.
eval_result = model.evaluate(
    input_fn=lambda:eval_input_fn(X_test, y_test, batch_size))

print('\nTest set accuracy: {accuracy:0.2f}\n'.format(**eval_result))

In [ ]:
eval_result

### Stacking

In [ ]:
# lr = LogisticRegression()
# stack = StackingClassifier(classifiers=[calibrated_rf, calibrated_GB], meta_classifier=lr)
# stack.fit(X_train, y_train)

## Predicting on validation set

In [ ]:
def score(y_true, y_pred):
    return log_loss(y_true, y_pred, eps=1e-16)

In [ ]:
preds = calibrated_rf.predict_proba(X_test)[:, 1]
score(y_test, preds)

In [ ]:
preds_GB = modelGB_fit.predict_proba(X_test)[:, 1]
score(y_test, preds_GB)

In [181]:
preds_xgb = xbg_model_fit.predict_proba(X_test)[:, 1]
score(y_test, preds_xgb)

0.40437752858318243

In [ ]:
predictions = list(model.predict(
    input_fn=lambda:eval_input_fn(X_test, y_test, batch_size)))

# Dictionary for predictions
col1 = []
col2 = []
col3 = []


for idx, input, p in zip(X_test.index, y_test, predictions):
    v  = p["class_ids"][0] 
    class_id = p['class_ids'][0]
    probability = p['probabilities'][1] # Probability
    
    # Adding to dataframe
    col1.append(idx) # Index
    col2.append(probability) # Prediction
    col3.append(input) # Expecter
    
   
    #print(template.format(idx, v, 100 * probability, input))


results = pd.DataFrame({'index':col1, 'prediction_proba':col2, 'expected':col3})
results.head()

In [ ]:
score(results.expected,results.prediction_proba)

In [ ]:
# preds_stack = stack.predict_proba(X_test)[:, 1]
# score(y_test, preds_stack)

### TEST SUBMISSION

In [182]:
def preprocess_feats(df, enc):
    feats = df.copy()
    # take the first age in the range and convert to integer
    feats["age_int"] = feats.age.str.slice(1, 3).astype(int)
    feats['body_site']=feats['body_site'].str.replace(' ','_').str.replace('/','_')
    X = pd.concat(
        [
            feats[["age_int", "sex","resolution"]],
            pd.DataFrame(
                enc.transform(feats[["melanoma_history","body_site"]]),
                columns=enc.get_feature_names_out(),
                index=feats.index,
            ),
        ],
        axis=1,
    )

    return X

processed_features = preprocess_feats(test_metadata, enc)

predsub = calibrated_xgb.predict_proba(processed_features)[:,1]

score(test_labels.relapse,predsub)

# predictions = list(model.predict(
#     input_fn=lambda:eval_input_fn(processed_features, test_labels.relapse, batch_size)))

# # Dictionary for predictions
# col1 = []
# col2 = []
# col3 = []


# for idx, input, p in zip(test_metadata.index, test_labels, predictions):
#     v  = p["class_ids"][0] 
#     class_id = p['class_ids'][0]
#     probability = p['probabilities'][1] # Probability
    
#     # Adding to dataframe
#     col1.append(idx) # Index
#     col2.append(probability) # Prediction
#     col3.append(input) # Expecter
    
   
#     #print(template.format(idx, v, 100 * probability, input))


# results = pd.DataFrame({'index':col1, 'prediction_proba':col2, 'expected':col3})
# results.head()

# score(results.expected,results.prediction_proba)

0.6973627538073585

# CODE SUBMISSION

In [ ]:
Path("submission/assets").mkdir(exist_ok=True, parents=True)

# save out model to assets directory in submission folder
dump(calibrated_rf, "submission/assets/random_forest_model.joblib")
dump(calibrated_GB, "submission/assets/gradient_boosting_model.joblib")
dump(xbg_model_fit, "submission/assets/xgboost_model.joblib")

# save out encoder to assets directory as well
dump(enc, "submission/assets/hotencoder.joblib")

In [ ]:
%%writefile submission/main.py

"""Solution for VisioMel Challenge"""

from joblib import load
from pathlib import Path

from loguru import logger
import numpy as np
import pandas as pd


DATA_ROOT = Path("/code_execution/data/")


def preprocess_feats(df, enc):
    feats = df.copy()
    # take the first age in the range and convert to integer
    feats["age_int"] = feats.age.str.slice(1, 3).astype(int)
    feats['body_site']=feats['body_site'].str.replace(' ','_').str.replace('/','_')
    X = pd.concat(
        [
            feats[["age_int", "sex","resolution"]],
            pd.DataFrame(
                enc.transform(feats[["melanoma_history","body_site"]]),
                columns=enc.get_feature_names_out(),
                index=feats.index,
            ),
        ],
        axis=1,
    )

    return X
    
    
def main():
    # load sumission format
    submission_format = pd.read_csv(DATA_ROOT / "submission_format.csv", index_col=0)
    
    # load test_metadata
    test_metadata = pd.read_csv(DATA_ROOT / "test_metadata.csv", index_col=0)
    
    logger.info("Loading feature encoder and model")
    calibrated_xgb = load("assets/xgboost_model.joblib")
    enc = load("assets/hotencoder.joblib")
    
    logger.info("Preprocessing features")
    processed_features = preprocess_feats(test_metadata, enc)
        
    logger.info("Checking test feature filenames are in the same order as the submission format")
    assert (processed_features.index == submission_format.index).all()
    
    logger.info("Checking test feature columns align with loaded model")
    assert (processed_features.columns == calibrated_xgb.feature_names_in_).all()
    
    logger.info("Generating predictions")
    submission_format["relapse"] = calibrated_xgb.predict_proba(processed_features)[:,1]

    # save as "submission.csv" in the root folder, where it is expected
    logger.info("Writing out submission.csv")
    submission_format.to_csv("submission.csv")


if __name__ == "__main__":
    main()

In [ ]:
# zip up submission
! cd submission; zip -r ../submission.zip ./*